In [1]:
import pandas as pd
import numpy as np
import pickle
import gc
import os
import time
import copy
import multiprocessing as mp
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score, log_loss
from tqdm import tqdm
from scipy import sparse, spatial
import warnings
warnings.filterwarnings('ignore')
tqdm.pandas()

/root/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict


In [2]:
import os
import tensorflow as tf

os.environ['CUDA_VISIBLE_DEVICES'] = "0"
config = tf.compat.v1.ConfigProto()
# config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
# session = tf.Session(config=config)

from keras.models import Sequential, Model
from keras.layers import Input, Dense, CuDNNLSTM, Bidirectional, Embedding, CuDNNGRU, Conv1D, MaxPooling1D, AveragePooling1D
from keras.layers import Flatten, PReLU, Dropout, BatchNormalization, SpatialDropout1D, concatenate
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing.sequence import pad_sequences
from keras.utils import Sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.engine.topology import Layer 
# from keras.utils.training_utils import multi_gpu_model

Using TensorFlow backend.


In [3]:
tic = time.time()
DATA_PATH = '../pkl/'
SAVE_PATH = './feats/'
WEIGHT_PATH = './weights/'
if not os.path.exists(SAVE_PATH):
    print('create dir: %s' % SAVE_PATH)
    os.mkdir(SAVE_PATH)
if not os.path.exists(WEIGHT_PATH):
    print('create dir: %s' % WEIGHT_PATH)
    os.mkdir(WEIGHT_PATH)

print('user_ques_topic_dnn_enc_A...')

user_ques_topic_dnn_enc_A...


In [4]:
data = pd.read_pickle(os.path.join(DATA_PATH, 'invite_data.pkl'))
user_info = pd.read_pickle(os.path.join(DATA_PATH, 'user_info.pkl'))
question_info = pd.read_pickle(os.path.join(DATA_PATH, 'question_info.pkl'))
topic = pd.read_pickle(os.path.join(DATA_PATH, 'topic.pkl'))
len_train = (~data['label'].isnull()).sum()
print(len_train)

9489162


In [5]:
data.head()

,iday,ihour,itime,label,qid,uid
0,3865,22,92782,0.0,2166419046,401693808
1,3844,11,92267,0.0,1550017551,3392373099
2,3862,15,92703,0.0,604029601,2317670257
3,3849,11,92387,0.0,2350061229,1618461867
4,3867,4,92812,0.0,2443223942,3544409350


In [6]:
user_info['utopic'] = user_info['topic_a'] + user_info['topic_ik']
question_info.rename(columns={'topic': 'qtopic'}, inplace=True)

In [7]:
data = data.merge(user_info[['uid', 'utopic']], 'left', 'uid').merge(question_info[['qid', 'qtopic']], 'left', 'qid')
# data['topic'] = data['topic'].fillna('-1')
data.head()

,iday,ihour,itime,label,qid,uid,utopic,qtopic
0,3865,22,92782,0.0,2166419046,401693808,"[1727, 5310, 3402, 916, 1506, 26329, 7293, 180...","[456, 112, 9566, 5310]"
1,3844,11,92267,0.0,1550017551,3392373099,"[42595, 3, 8520, 597, 6485, 6212, 25664, 148, ...","[2, 3095]"
2,3862,15,92703,0.0,604029601,2317670257,"[610, 448, 61, 2801, 9019, 65, 233, 190, 55, 5...","[6090, 2156, 97, 456]"
3,3849,11,92387,0.0,2350061229,1618461867,"[5, 33331, 2274, 31, 245, 516, 309, 1326, 119,...",[856]
4,3867,4,92812,0.0,2443223942,3544409350,"[0, 4876, 2467, 245, 68, 556, 42, 8, 825, 227,...","[26, 76, 17]"


In [8]:
max_len_u = data['utopic'].apply(len).max()
max_len_q = data['qtopic'].apply(len).max()
print(max_len_u, max_len_q) # 110

110 13


In [9]:
np.random.seed(42)
def shuffle(l):
    np.random.shuffle(l)
    return l

data['utopic'] = data['utopic'].progress_apply(shuffle)
data['qtopic'] = data['qtopic'].progress_apply(shuffle)

100%|██████████| 10630845/10630845 [00:19<00:00, 541222.80it/s]


In [10]:
data.head()

,iday,ihour,itime,label,qid,uid,utopic,qtopic
0,3865,22,92782,0.0,2166419046,401693808,"[16, 1734, 18098, 2794, 14572, 38, 3020, 28, 1...","[5310, 112, 9566, 456]"
1,3844,11,92267,0.0,1550017551,3392373099,"[25664, 2959, 394, 460, 2294, 910, 112, 8520, ...","[3095, 2]"
2,3862,15,92703,0.0,604029601,2317670257,"[316, 207, 2801, 8943, 148, 553, 233, 11078, 9...","[97, 6090, 456, 2156]"
3,3849,11,92387,0.0,2350061229,1618461867,"[1074, 320, 124, 1918, 119, 31, 820, 1096, 177...",[856]
4,3867,4,92812,0.0,2443223942,3544409350,"[825, 4876, 556, 0, 68, 637, 42, 2467, 245, 22...","[17, 76, 26]"


In [11]:
qid_topic_map_32 = data[['qid', 'qtopic']].copy()
qid_topic_map_32.drop_duplicates(subset=['qid'], inplace=True)
qid_topic_map_32.to_pickle('./shuffle_map/qid_topic_map_32.pkl')
print(qid_topic_map_32.shape)

(1083727, 2)


In [13]:
uid_topic_map_32 = data[['uid', 'utopic']].copy()
uid_topic_map_32.drop_duplicates(subset=['uid'], inplace=True)
uid_topic_map_32.to_pickle('./shuffle_map/uid_topic_map_32.pkl')
print(uid_topic_map_32.shape)

(1419265, 2)


In [11]:
utopics = pad_sequences(data['utopic'], maxlen=max_len_u)
qtopics = pad_sequences(data['qtopic'], maxlen=max_len_q)

In [12]:
utopics.shape

(10630845, 110)

In [13]:
embed_weights = np.array([[0] * 64] + [list(v) for v in topic['vector'].values])

In [14]:
embed_weights.shape

(100001, 64)

In [15]:
class DataSequence(Sequence):
    
    def __init__(self, xu, xq, y, batch_size=128):
        self.xu = xu
        self.xq = xq
        self.y = y
        self.batch_size = batch_size
         
    def __len__(self):
        return int(np.ceil(self.xu.shape[0] / float(self.batch_size)))
    
    def __getitem__(self, idx):
        batch_idx = np.arange(idx * self.batch_size, min((idx + 1) * self.batch_size, self.xu.shape[0]))
        batch_xu = self.xu[batch_idx]
        batch_xq = self.xq[batch_idx]
        batch_y = self.y[batch_idx]
        return [batch_xu, batch_xq], batch_y

In [16]:
class MetricsCallback(Callback):
    def __init__(self, x_trn_u, x_trn_q, y_trn, x_val_u, x_val_q, y_val, batch_size=128, save_name='weight.h5'):
        self.trn_generator = DataSequence(x_trn_u, x_trn_q, y_trn, BATCH_SIZE)
        self.val_generator = DataSequence(x_val_u, x_val_q, y_val, BATCH_SIZE)
        self.y_trn = y_trn
        self.y_val = y_val
        self.save_name = save_name
        self.best_score = 0.5

    def on_epoch_end(self, epoch, logs={}):
        # eval train
        y_pred = self.model.predict_generator(self.trn_generator, 
                                              max_queue_size=10, 
                                              workers=1, 
                                              use_multiprocessing=False, 
                                              verbose=0)
        roc = roc_auc_score(self.y_trn, y_pred)
        # eval valid
        y_pred_val = self.model.predict_generator(self.val_generator, 
                                              max_queue_size=10, 
                                              workers=1, 
                                              use_multiprocessing=False, 
                                              verbose=0)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
#         print(roc_val)
#         print(log_loss(self.y_val, y_pred_val)/len(y_pred_val))
        if roc_val > self.best_score:
            self.best_score = roc_val
            self.model.save_weights(os.path.join(WEIGHT_PATH, self.save_name))
        
        return

In [17]:
def Net(hiddens=(256,)):
    # user topic
    inp_u = Input(shape=(max_len_u,))
    x = Embedding(embed_weights.shape[0], embed_weights.shape[1], weights=[embed_weights], trainable=False)(inp_u)
    x = Dropout(0.2)(x)
    x = Conv1D(256, 2, padding='valid', activation='relu', strides=1)(x)
    x = AveragePooling1D(pool_size=4)(x)
    x = Flatten()(x)
    # ques topic
    inp_q = Input(shape=(max_len_q,))
    y = Embedding(embed_weights.shape[0], embed_weights.shape[1], weights=[embed_weights], trainable=False)(inp_q)
    y = Dropout(0.2)(y)
    y = Conv1D(256, 2, padding='valid', activation='relu', strides=1)(y)
    y = AveragePooling1D(pool_size=4)(y)
    y = Flatten()(y)
    # concat
    x = concatenate([x, y], axis=1)
    for hs in hiddens:
        x = Dense(hs)(x)
        x = BatchNormalization()(x)
        x = PReLU()(x)
        x = Dropout(0.2)(x)
    out = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=[inp_u, inp_q], outputs=out)
    return model

In [18]:
train_x_u = utopics[:len_train]
train_x_q = qtopics[:len_train]
test_x_u = utopics[len_train:]
test_x_q = qtopics[len_train:]
train_y = data['label'].values[:len_train]

In [19]:
print(train_x_u.shape)
print(test_x_u.shape)
print(train_y.sum() / len(train_y))

(9489162, 110)
(1141683, 110)
0.1773515933229931


In [20]:
BATCH_SIZE = 1024

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# for i, (tr_idx, va_idx) in enumerate(kfold.split(train_x_u, train_y)):
#     print('-'*100)
#     print('Fold %d' % i)
#     trn_x_u, trn_x_q, trn_y = train_x_u[tr_idx], train_x_q[tr_idx], train_y[tr_idx]
#     val_x_u, val_x_q, val_y = train_x_u[va_idx], train_x_q[va_idx], train_y[va_idx]
    
#     K.clear_session()
#     model = Net(hiddens=(256,256,256))
#     model.compile(loss='binary_crossentropy', optimizer='adam',)
    
#     trn_generator = DataSequence(trn_x_u, trn_x_q, trn_y, batch_size=BATCH_SIZE)
#     val_generator = DataSequence(val_x_u, val_x_q, val_y, batch_size=BATCH_SIZE)
    
#     history = model.fit_generator(generator=trn_generator, 
#                         epochs=30, 
#                         verbose=1, 
#                         callbacks=[MetricsCallback(trn_x_u, trn_x_q, trn_y, val_x_u, val_x_q, val_y, 
#                                                    batch_size=BATCH_SIZE*4, 
#                                                    save_name='user_ques_topic_dnn_enc_A_weight_fold%d.h5' % i)], 
#                         max_queue_size=100, 
#                         workers=1, 
#                         use_multiprocessing=False,)
    
#     del trn_x_u, trn_x_q, trn_y, val_x_u, val_x_q, val_y
#     gc.collect()

In [21]:
# 0.4608

In [21]:
test_generator = DataSequence(test_x_u, test_x_q, np.zeros(test_x_u.shape[0]), batch_size=BATCH_SIZE * 4)

test_pred = np.zeros((test_x_u.shape[0], 1))
train_pred = np.zeros((train_x_u.shape[0], 1))

for i, (tr_idx, va_idx) in enumerate(kfold.split(train_x_u, train_y)):
    print('-'*100)
    print('Fold %d' % i)
    trn_x_u, trn_x_q, trn_y = train_x_u[tr_idx], train_x_q[tr_idx], train_y[tr_idx]
    val_x_u, val_x_q, val_y = train_x_u[va_idx], train_x_q[va_idx], train_y[va_idx]
    
    K.clear_session()
    model = Net(hiddens=(256,256,256))
    model.compile(loss='binary_crossentropy', optimizer='adam',)
    
    trn_generator = DataSequence(trn_x_u, trn_x_q, trn_y, batch_size=BATCH_SIZE)
    val_generator = DataSequence(val_x_u, val_x_q, val_y, batch_size=BATCH_SIZE)
    
    model.load_weights(os.path.join(WEIGHT_PATH, 'user_ques_topic_dnn_enc_A_weight_fold%d.h5' % i))
    
    train_pred[va_idx] = model.predict_generator(val_generator)
    test_pred += model.predict_generator(test_generator) / 5
    
    print(roc_auc_score(val_y, train_pred[va_idx]))
    
    del trn_x_u, trn_x_q, trn_y, val_x_u, val_x_q, val_y
    gc.collect()
    print('Predict Done.')

----------------------------------------------------------------------------------------------------
Fold 0
0.6873482322756785
Predict Done.
----------------------------------------------------------------------------------------------------
Fold 1
0.6867734941994806
Predict Done.
----------------------------------------------------------------------------------------------------
Fold 2
0.6876299787087984
Predict Done.
----------------------------------------------------------------------------------------------------
Fold 3
0.6865914388143324
Predict Done.
----------------------------------------------------------------------------------------------------
Fold 4
0.6862721115192466
Predict Done.


In [22]:
arr = np.vstack([train_pred, test_pred])
feat = pd.DataFrame(arr, columns=['user_ques_topic_dnn_enc_A'])
feat.head()

,user_ques_topic_dnn_enc_A
0,0.083848
1,0.082480
2,0.076444
3,0.032672
4,0.179217


In [23]:
feat.to_pickle('./feats/user_ques_topic_dnn_enc_A.pkl')

In [24]:
feat.shape

(10630845, 1)